In [134]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolAlign
import RA
import RR
import py_rdl


In [105]:
# Construct a 6-membered ring--cyclohexane
mol0 = Chem.MolFromSmiles("C1CCCCC1")
AllChem.EmbedMolecule(mol0)

0

In [106]:
# Initialize bond length and bond angle
# C-C 1.52 angstrom
# C-C-C 1.93 radian
bl = [1.52]*6 
bang = [1.93]*6

# RR.SetRingPuckerCoords(mol, ring atom idx, [amplitudes], [phase angles], bondlength, bondangle)
# Example -- we fix to a chair conformation, but with different phase angle 
coord1 = RR.SetRingPuckerCoords(mol0, list(range(6)),[0.021243,0.589856], [1.959] ,  bl, bang) # chair
coord2 = RR.SetRingPuckerCoords(mol0, list(range(6)),[0.60,0.002], [np.pi], bl, bang) # boat

In [116]:
for j in range(6):
    mol0.GetConformer().SetAtomPosition(j,coord1[j])
mol0.AddConformer(mol0.GetConformer(),assignId=1)

for j in range(6):
    mol0.GetConformer().SetAtomPosition(j,coord2[j])
mol0.AddConformer(mol0.GetConformer(),assignId=2)


3

In [131]:
# RMSD between boat and chair conformation
AllChem.GetConformerRMS(mol0,1,2)

0.35170479518311837

In [138]:
# Ring Conformational Anaylsis

# Get Bond Set
bonds = []
for bond in mol0.GetBonds():
    bonds.append((bond.GetBeginAtom().GetIdx(),bond.GetEndAtom().GetIdx()))

# Apply RingDecomposerLib to compute RCs/URFs
data = py_rdl.Calculator.get_calculated_result(bonds)
for urf in data.urfs:
    rcs = data.get_relevant_cycles_for_urf(urf)
    for rc in rcs:
        ringloop = RA.Rearrangement(mol0, list(rc.nodes)) # rearrange the ring atom order 
        coord = np.array([mol0.GetConformer(1).GetAtomPosition(atom) for atom in ringloop]) # get current ring atom coordinates
        ccoord = RA.Translate(coord) # translate ring with origin as cetner
        cremerpople = RA.GetRingPuckerCoords(ccoord) # get cremer-pople parameters

# check if we get the same parameter as we set previously
print(cremerpople) 

([0.5999999999999998, 0.00200000000000015], [3.141592653589793])


In [142]:
# Compute ring substituent position
# Example from Manuscript (Figure 2)
methylcyclohexane = Chem.SDMolSupplier("example.sdf")[0]

# Ring Atom Index 1,2,3,4,5,6  Methyl Substituent Atom Index 0
RA.GetRingSubstituentPosition(methylcyclohexane, [1,2,3,4,5,6], [1,0]) # methyl carbon 0 is attached to ring atom 1 

(0.23886741752156593, -2.2538930379170905)